In [1]:
from pathlib import Path
import sys
import pandas as pd

from pathlib import Path
ROOT = Path.cwd().resolve().parents[0]

sys.path.insert(0, str(ROOT))

from src.io_xfoil import parse_xfoil_polar_txt

p1 = ROOT / "data" / "raw" / "naca2412_Re1e6_M0p2_polar_0p5deg.txt"
p2 = ROOT / "data" / "raw" / "re2e6_0p5"  # (ohne Endung ist OK)

print("p1 exists:", p1.exists(), "size:", p1.stat().st_size)
print("p2 exists:", p2.exists(), "size:", p2.stat().st_size)

df1 = parse_xfoil_polar_txt(p1)
df2 = parse_xfoil_polar_txt(p2)

print("df1 rows:", len(df1), "unique alphas:", df1["alpha_deg"].nunique(), "Re:", df1["Re"].iloc[0])
print("df2 rows:", len(df2), "unique alphas:", df2["alpha_deg"].nunique(), "Re:", df2["Re"].iloc[0])

df = pd.concat([df1, df2], ignore_index=True)
df = df.sort_values(["Re", "alpha_deg"]).reset_index(drop=True)

out_csv = ROOT / "data" / "processed" / "xfoil_polars.csv"
df.to_csv(out_csv, index=False)

print("\nWROTE:", out_csv, "rows:", len(df), "unique alphas:", df["alpha_deg"].nunique())
print(df.groupby(["Re","Mach"]).size())


p1 exists: True size: 5777
p2 exists: True size: 3005
df1 rows: 81 unique alphas: 41 Re: 1000000.0
df2 rows: 39 unique alphas: 39 Re: 2000000.0

WROTE: C:\Users\aravi\PycharmProjects\aero-surrogate\data\processed\xfoil_polars.csv rows: 120 unique alphas: 41
Re         Mach
1000000.0  0.2     81
2000000.0  0.2     39
dtype: int64
